--- Day 7: No Space Left On Device ---


You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
cd / switches the current directory to the outermost directory, /.
ls means list. It prints out all of the files and directories immediately contained by the current directory:
123 abc means that the current directory contains a file named abc with size 123.
dir xyz means that the current directory contains a directory named xyz.
Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
Directory d has total size 24933642.
As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.
To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [36]:
with open('../data/day7.txt') as f:
    input_data = f.read()
    
input_commands = [command.replace('\n', ' ').strip().split() for command in input_data.split('$')[2:]]

In [4]:
class Tree:
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.children = []   
        
    def get_size(self):
        return sum([child.get_size() for child in self.children])
    
    def add_child(self, data, parent):
        if data[0] == 'dir':
            self.children.append(Tree(data[1], parent))
        else:
            self.children.append(File(data[1], int(data[0])))
        return self
    
    def move(self, command):
        if command == '..':
            return self.get_parent()
        else:
            child = self.move_to_child(command)
            if child:
                return child
            else: 
                logger.error(f'{command} not a child of {self.name}')
    
    def get_parent(self):
        return self.parent  
    
    def has_child(self, name):
        return name in [child.name for child in self.children]
    
    def move_to_child(self, name):
        return next((child for child in self.children if child.name == name), None)
    
    def move_to_root(self):
        while self.parent:
            self = self.get_parent()
        return self
        

class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size
    
    def get_size(self):
        return self.size

In [53]:
from loguru import logger

tree = Tree('/', None)


for log in input_commands:
    if log[0] == 'cd':
        tree = tree.move(log[1])
    elif log[0] == 'ls':
        log = log[1:]
        for i in range(0,len(log), 2):
            tree = tree.add_child((log[i], log[i+1]), tree) 
    else:
        logger.error(f'Command "{log[1]}" not known.')
    

In [58]:
def recursive_size_getter(tree, dir_dict):
    for child in tree.children:
        if hasattr(child, 'children'): #is folder
            dir_dict = recursive_size_getter(child, dir_dict)  
    dir_dict[tree.name] = tree.get_size()
    return dir_dict
    
dir_dict = {}  
tree = tree.move_to_root()
dir_sizes = recursive_size_getter(tree, dir_dict)   

In [59]:
sum([dir_size for dir_name, dir_size in dir_sizes.items() if dir_size < 100_000]) 

1206923

In [8]:
#1206923

'/'

In [37]:
len(dir_sizes.values())

143

In [42]:
dir_sizes

{'dngldfww': 102218,
 'dzplphqw': 263191,
 'mvslzl': 365409,
 'nnlzrwgh': 141976,
 'svc': 205836,
 'wshf': 25072,
 'dhbrmn': 652883,
 'wvfdjv': 104669,
 'rcdwft': 104669,
 'wtmdzgvg': 1099415,
 'qdtls': 1815835,
 'brhvclj': 60019,
 'clnvqg': 34772236,
 'pqp': 409771,
 'tdvtcmwz': 100584,
 'qbms': 456639,
 'wshwhzw': 57458,
 'dfmgqmvm': 775231,
 'ttntt': 109641,
 'rlbnv': 55214,
 'wfcczs': 55214,
 'tfns': 2267101,
 'sgbwd': 267083,
 'bchstbpm': 510366,
 'dpnqrg': 140379,
 'gsvwsc': 84951,
 'rfcssr': 13174,
 'cpbh': 13930,
 'drgcn': 641306,
 'mrv': 1374480,
 'vhjtgv': 98550,
 'zgtnfm': 98550,
 'jshpz': 2806903,
 'bgbdbr': 2806903,
 'qqr': 143274,
 'gsplfsvm': 103196,
 'zzhwq': 103196,
 'qwwl': 411742,
 'cjcpzv': 173344,
 'gcnc': 1525047,
 'bddbffsf': 181609,
 'bpgmc': 122736,
 'nrrpsn': 58623,
 'flz': 249582,
 'ddrthl': 333778,
 'pmgzdp': 94671,
 'qgvzbclg': 183904,
 'nhjthr': 383644,
 'dptjfd': 663986,
 'hqncdcdh': 218709,
 'ddbcvhqr': 2195372,
 'gsfzgvpz': 266749,
 'dlmhm': 50087,
 'mp